In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import classification_report
import missingno as msno
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv("~/Downloads/heart.csv", delimiter=",")

In [5]:
df.columns

Index(['HeartDisease', 'BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
       'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
       'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime',
       'Asthma', 'KidneyDisease', 'SkinCancer'],
      dtype='object')

In [7]:
df_zero = df[df["SkinCancer"] == "No"]

# Sélectionner aléatoirement 20 000 lignes
df_zero_sample = df_zero.sample(n=15891, random_state=42)

In [8]:
df_one = df[df["SkinCancer"] == "Yes"]

In [9]:
df_balanced = pd.concat([df_zero_sample, df_one], axis=0)

In [10]:
df_balanced

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
122059,No,54.53,No,No,No,3.0,2.0,Yes,Female,55-59,Black,No,No,Very good,5.0,Yes,No,No
52848,No,31.75,Yes,No,No,0.0,0.0,No,Female,70-74,White,No,Yes,Very good,7.0,No,No,No
22543,No,32.01,No,No,No,0.0,0.0,No,Male,45-49,Hispanic,No,Yes,Good,8.0,No,No,No
280863,No,24.81,No,No,No,0.0,0.0,No,Male,60-64,White,No,Yes,Good,7.0,No,No,No
83700,No,24.33,No,No,No,0.0,0.0,No,Female,30-34,White,No,Yes,Excellent,7.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319399,No,30.91,No,No,No,0.0,0.0,Yes,Female,65-69,Hispanic,No,No,Fair,6.0,Yes,No,Yes
319431,No,26.63,Yes,No,No,7.0,30.0,No,Female,55-59,Hispanic,Yes,Yes,Fair,6.0,No,No,Yes
319454,Yes,24.78,Yes,No,No,0.0,15.0,No,Male,75-79,Hispanic,No,No,Fair,5.0,Yes,No,Yes
319523,No,26.57,Yes,No,No,30.0,0.0,Yes,Female,55-59,Hispanic,No,Yes,Excellent,8.0,No,No,Yes


In [11]:
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
df=df_balanced

In [13]:
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,29.76,No,No,No,0.0,0.0,No,Male,65-69,White,Yes,No,Good,9.0,Yes,No,Yes
1,No,27.37,Yes,No,No,0.0,0.0,No,Male,80 or older,White,No,Yes,Good,8.0,No,No,Yes
2,No,26.40,No,No,No,1.0,7.0,No,Male,55-59,White,No,Yes,Good,4.0,Yes,No,Yes
3,No,21.95,No,No,Yes,0.0,0.0,No,Female,65-69,Asian,Yes (during pregnancy),Yes,Good,8.0,Yes,No,Yes
4,No,28.19,No,No,No,0.0,0.0,No,Female,60-64,Hispanic,No,No,Very good,8.0,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45705,No,22.81,Yes,No,No,30.0,30.0,Yes,Female,55-59,White,Yes,No,Poor,4.0,No,Yes,No
45706,No,23.78,No,No,No,0.0,0.0,No,Female,70-74,White,No,Yes,Very good,8.0,No,No,Yes
45707,No,28.21,No,No,No,0.0,0.0,No,Male,65-69,White,No,Yes,Very good,8.0,No,No,Yes
45708,No,17.38,No,No,No,0.0,4.0,No,Female,18-24,White,No,Yes,Good,9.0,No,No,No


In [14]:
df.describe()

,BMI,PhysicalHealth,MentalHealth,SleepTime
count,45710.000000,45710.000000,45710.000000,45710.000000
mean,27.913595,4.039422,3.439947,7.209735
std,5.909957,8.754404,7.663158,1.425541
min,12.130000,0.000000,0.000000,1.000000
25%,23.910000,0.000000,0.000000,6.000000
50%,27.065000,0.000000,0.000000,7.000000
75%,30.850000,2.000000,2.000000,8.000000
max,85.910000,30.000000,30.000000,24.000000


In [15]:
df.describe(include="object")

,HeartDisease,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,Asthma,KidneyDisease,SkinCancer
count,45710,45710,45710,45710,45710,45710,45710,45710,45710,45710,45710,45710,45710,45710
unique,2,2,2,2,2,2,13,6,4,2,5,2,2,2
top,No,No,No,No,No,Female,80 or older,White,No,Yes,Very good,No,No,Yes
freq,39484,25429,42637,43170,37406,23342,7190,40454,37419,35382,16149,39619,43028,29819


In [22]:
df["GenHealth"].unique()

array(['Good', 'Very good', 'Excellent', 'Poor', 'Fair'], dtype=object)

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [27]:
data=df

In [29]:
binary_cols = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 
               'Sex', 'Diabetic', 'PhysicalActivity', 'Asthma', 'KidneyDisease']

# Colonnes ordinales
genhealth_categories = ['Poor', 'Fair', 'Good', 'Very good', 'Excellent']
age_categories = ['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59',
                  '60-64','65-69','70-74','75-79','80 or older']

ordinal_cols = ['GenHealth', 'AgeCategory']

# Colonnes à one-hot encoder
onehot_cols = ['Race']

# Colonnes numériques à scaler
numeric_cols = ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']

# Préparation du transformer pour les colonnes binaires (Yes/No => 1/0)
def binary_map(x):
    if x in ['Yes', 'Male', 'Yes (during pregnancy)']:
        return 1
    else:
        return 0

# Preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('binary', Pipeline([
        ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))
    ]), binary_cols),
    ('ordinal', OrdinalEncoder(categories=[genhealth_categories, age_categories]), ordinal_cols),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), onehot_cols),
    ('scale', StandardScaler(), numeric_cols)
])

# Pipeline complet
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression())
])

# Définir X et y
X = data.drop('SkinCancer', axis=1)
y = data['SkinCancer'].apply(lambda x: 1 if x=='Yes' else 0)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)
print(y_pred)

/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


[1 1 1 ... 1 1 1]


/home/sebabte/kraya/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


In [30]:
X

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease
0,No,29.76,No,No,No,0.0,0.0,No,Male,65-69,White,Yes,No,Good,9.0,Yes,No
1,No,27.37,Yes,No,No,0.0,0.0,No,Male,80 or older,White,No,Yes,Good,8.0,No,No
2,No,26.40,No,No,No,1.0,7.0,No,Male,55-59,White,No,Yes,Good,4.0,Yes,No
3,No,21.95,No,No,Yes,0.0,0.0,No,Female,65-69,Asian,Yes (during pregnancy),Yes,Good,8.0,Yes,No
4,No,28.19,No,No,No,0.0,0.0,No,Female,60-64,Hispanic,No,No,Very good,8.0,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45705,No,22.81,Yes,No,No,30.0,30.0,Yes,Female,55-59,White,Yes,No,Poor,4.0,No,Yes
45706,No,23.78,No,No,No,0.0,0.0,No,Female,70-74,White,No,Yes,Very good,8.0,No,No
45707,No,28.21,No,No,No,0.0,0.0,No,Male,65-69,White,No,Yes,Very good,8.0,No,No
45708,No,17.38,No,No,No,0.0,4.0,No,Female,18-24,White,No,Yes,Good,9.0,No,No


In [35]:
y_pred[4]

np.int64(1)

In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# Prédictions sur le jeu de test
y_pred = pipeline.predict(X_test)

# Calcul des métriques
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

# Optionnel : rapport complet
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Precision: 0.782
Recall: 0.914
F1-score: 0.842

Classification Report:

              precision    recall  f1-score   support

           0       0.77      0.53      0.63      3222
           1       0.78      0.91      0.84      5920

    accuracy                           0.78      9142
   macro avg       0.78      0.72      0.74      9142
weighted avg       0.78      0.78      0.77      9142



/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


In [37]:
from sklearn.ensemble import RandomForestClassifier

pipeline_rf = Pipeline([
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)

print("=== Random Forest ===")
print(classification_report(y_test, y_pred_rf))


/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))
/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


=== Random Forest ===
              precision    recall  f1-score   support

           0       0.69      0.55      0.61      3222
           1       0.78      0.87      0.82      5920

    accuracy                           0.76      9142
   macro avg       0.74      0.71      0.72      9142
weighted avg       0.75      0.76      0.75      9142



In [38]:
from sklearn.ensemble import GradientBoostingClassifier

pipeline_gb = Pipeline([
    ('preprocess', preprocessor),
    ('clf', GradientBoostingClassifier(n_estimators=100, random_state=42))
])

pipeline_gb.fit(X_train, y_train)
y_pred_gb = pipeline_gb.predict(X_test)

print("=== Gradient Boosting ===")
print(classification_report(y_test, y_pred_gb))


/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


=== Gradient Boosting ===
              precision    recall  f1-score   support

           0       0.79      0.51      0.62      3222
           1       0.78      0.93      0.84      5920

    accuracy                           0.78      9142
   macro avg       0.78      0.72      0.73      9142
weighted avg       0.78      0.78      0.76      9142



/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


In [39]:
from sklearn.neighbors import KNeighborsClassifier

pipeline_knn = Pipeline([
    ('preprocess', preprocessor),
    ('clf', KNeighborsClassifier(n_neighbors=5))
])

pipeline_knn.fit(X_train, y_train)
y_pred_knn = pipeline_knn.predict(X_test)

print("=== K-Nearest Neighbors ===")
print(classification_report(y_test, y_pred_knn))


/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))
/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


=== K-Nearest Neighbors ===
              precision    recall  f1-score   support

           0       0.68      0.52      0.59      3222
           1       0.77      0.86      0.81      5920

    accuracy                           0.74      9142
   macro avg       0.72      0.69      0.70      9142
weighted avg       0.74      0.74      0.74      9142



In [40]:
import joblib

In [43]:
# Exemple pour Logistic Regression
joblib.dump(pipeline_lr, 'pipeline_logistic_regression.pkl')

# Random Forest
joblib.dump(pipeline_rf, 'pipeline_random_forest.pkl')

# Gradient Boosting
joblib.dump(pipeline_gb, 'pipeline_gradient_boosting.pkl')

# K-Nearest Neighbors
joblib.dump(pipeline_knn, 'pipeline_knn.pkl')


PicklingError: Can't pickle <function <lambda> at 0x76e7e5b8a8e0>: it's not found as __main__.<lambda>

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

pipeline_lr = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

pipeline_lr.fit(X_train, y_train)
y_pred_lr = pipeline_lr.predict(X_test)

print("=== Logistic Regression ===")
print(classification_report(y_test, y_pred_lr))


/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.77      0.53      0.63      3222
           1       0.78      0.91      0.84      5920

    accuracy                           0.78      9142
   macro avg       0.78      0.72      0.74      9142
weighted avg       0.78      0.78      0.77      9142



/tmp/ipykernel_32204/3190021452.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ('mapper', FunctionTransformer(lambda df: df.applymap(binary_map)))


In [44]:
binary_cols = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 
               'Sex', 'Diabetic', 'PhysicalActivity', 'Asthma', 'KidneyDisease']

genhealth_categories = ['Poor', 'Fair', 'Good', 'Very good', 'Excellent']
age_categories = ['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59',
                  '60-64','65-69','70-74','75-79','80 or older']

ordinal_cols = ['GenHealth', 'AgeCategory']
onehot_cols = ['Race']
numeric_cols = ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']

# ------------------------------
# Fonction binaire nommée pour FunctionTransformer
# ------------------------------
def binary_map(x):
    if x in ['Yes', 'Male', 'Yes (during pregnancy)']:
        return 1
    else:
        return 0

def binary_transform(df):
    return df.applymap(binary_map)

# ------------------------------
# Préprocessing
# ------------------------------
preprocessor = ColumnTransformer(transformers=[
    ('binary', FunctionTransformer(binary_transform), binary_cols),
    ('ordinal', OrdinalEncoder(categories=[genhealth_categories, age_categories]), ordinal_cols),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), onehot_cols),
    ('scale', StandardScaler(), numeric_cols)
])

# ------------------------------
# Définir X et y
# ------------------------------
X = data.drop('SkinCancer', axis=1)
y = data['SkinCancer'].apply(lambda x: 1 if x=='Yes' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------------------
# Liste des modèles
# ------------------------------
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# ------------------------------
# Entraînement, évaluation et sauvegarde
# ------------------------------
for name, model in models.items():
    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('clf', model)
    ])
    
    # Entraînement
    pipeline.fit(X_train, y_train)
    
    # Prédictions
    y_pred = pipeline.predict(X_test)
    
    print(f"=== {name} ===")
    print(classification_report(y_test, y_pred))
    
    # Sauvegarde
    filename = f'pipeline_{name.replace(" ", "_").lower()}.pkl'
    joblib.dump(pipeline, filename)
    print(f"Modèle sauvegardé sous : {filename}\n")

/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)
/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)
/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)


=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.77      0.53      0.63      3222
           1       0.78      0.91      0.84      5920

    accuracy                           0.78      9142
   macro avg       0.78      0.72      0.74      9142
weighted avg       0.78      0.78      0.77      9142

Modèle sauvegardé sous : pipeline_logistic_regression.pkl



/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)


=== Random Forest ===
              precision    recall  f1-score   support

           0       0.69      0.55      0.61      3222
           1       0.78      0.87      0.82      5920

    accuracy                           0.76      9142
   macro avg       0.74      0.71      0.72      9142
weighted avg       0.75      0.76      0.75      9142

Modèle sauvegardé sous : pipeline_random_forest.pkl



/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)
/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)
/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)


=== Gradient Boosting ===
              precision    recall  f1-score   support

           0       0.79      0.51      0.62      3222
           1       0.78      0.93      0.84      5920

    accuracy                           0.78      9142
   macro avg       0.78      0.72      0.73      9142
weighted avg       0.78      0.78      0.76      9142

Modèle sauvegardé sous : pipeline_gradient_boosting.pkl



/tmp/ipykernel_32204/2758211053.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(binary_map)


=== KNN ===
              precision    recall  f1-score   support

           0       0.68      0.52      0.59      3222
           1       0.77      0.86      0.81      5920

    accuracy                           0.74      9142
   macro avg       0.72      0.69      0.70      9142
weighted avg       0.74      0.74      0.74      9142

Modèle sauvegardé sous : pipeline_knn.pkl



In [45]:
import os 
os.getcwd()

'/home/sebabte/cancer_app'

In [46]:
joblib.__version__

'1.5.2'

In [51]:
df["AgeCategory"].unique()

array(['65-69', '80 or older', '55-59', '60-64', '70-74', '50-54',
       '18-24', '75-79', '35-39', '30-34', '40-44', '45-49', '25-29'],
      dtype=object)